In [1]:
# This cell is removed with the tag: "remove-input"
# As such, it will not be shown in documentation

# import warnings
# warnings.filterwarnings('ignore')

# Working with NGLView

MolSysMT can handle nglview.NGLWidget objects in the same way that it handles other molecular systems. Therefore, any MolSysMT tool can accept NGLView views as input systems.

## The nglview.NGLWidget as any other molecular system's form

Before we show some examples of how MolSysMT works with NGLView native objects, let's create an nglview.NGLWidget to play with. Let's work with the demo system from NGLView:

In [2]:
import molsysmt as msm
import nglview as nv

In [3]:
view = nv.demo()

In [4]:
view

NGLWidget()

Now that we have an nglview.NGLWidget object, let's see if MolSysMT can get an information summary of the molecular system straight from it as first test:

In [5]:
msm.info(view)

ValueError: invalid literal for int() with base 10: '   '

Attributes such as atom names, number of structures or atom coordinates can be retrieved with the function {func}`molsysmt.basic.get`:

In [ ]:
msm.get(view, element='group', selection=[81, 82, 83], name=True)

In [ ]:
msm.get(view, element='system', n_structures=True)

In [ ]:
msm.get(view, element='atom', selection='atom_name=="CA"', coordinates=True)

Element's selections can also be made on NGLView native objects with MolSysMT:

In [ ]:
msm.select(view, selection='atom_name=="CA" and group_name=="LYS"')

And in addition, MolSysMT can help you to get selection strings to be used in NGLView:

In [ ]:
msm.select(view, selection='atom_name=="CA" and group_name=="LYS"', to_syntax='nglview')

In [ ]:
msm.select(view, element='group', selection='group_name=="LYS"', to_syntax='nglview')

Finally we can wonder, can MolSysMT convert an nglview.NGLWidget to other molecular system forms? Have a look to this couple of examples:

In [ ]:
msm.convert(view, to_form='string:aminoacids3')

In [ ]:
openmm_Topology = msm.convert(view, to_form='openmm.Topology')
msm.info(openmm_Topology)


Now that it has been demonstrated that MolSysMT recognizes nglview.NGLWidget objects as any other molecular form. Let's show a couple of examples more appealing than the previous cells.

## Example 1: Getting contact maps from a trajectory view

Given the visualization of a molecular dynamics trajectory, let's compute the contact maps between all alpha carbons from all frames.

First, the view needs to be produced by NGLView. This let's do it straight with MolSysMT:

In [ ]:
view = msm.convert([nv.datafiles.GRO, nv.datafiles.XTC], to_form='nglview.NGLWidget')

:::{note}
We could have used the function :func:`molsysmt.basic.view` instead also. Have a look to [User guide > Tools > Basic > View](../tools/basic/view.ipynb).
:::

In [ ]:
msm.info(view)

Let's get the contact maps from the 51 structures with a 12 angstroms threshold:

In [ ]:
contact_map = msm.structure.get_contacts(view, selection='molecule_type=="protein" and atom_name=="CA"', threshold='12 angstroms')

Each contact map is a boolean matrix:

In [ ]:
contact_map[10]

We can make with Plotly an animated representation of these contact maps. But before, let's get the x and y ticks labels for our contacts:

In [ ]:
CA_labels = msm.get_label(view, selection='molecule_type=="protein" and atom_name=="CA"', string='{group_name}-{group_id}')

In [ ]:
CA_labels[10]

In [ ]:
# This cell is skipped with the cell tag "remove-cell"

#%matplotlib widget # This in the first cell of the notebook

import matplotlib.pyplot as plt
import matplotlib.animation as animation


plt.rcParams["animation.html"] = "jshtml"

fig = plt.figure()

structure_index = 0
im = plt.imshow(contact_map[structure_index], cmap='Greys', origin='lower', animated=True)
plt.xticks(range(0,contact_map.shape[1],25),CA_labels[::25], rotation=45)
plt.yticks(range(0,contact_map.shape[1],25),CA_labels[::25], rotation=45)

def updatefig(ii):
    global structure_index, contact_map
    structure_index = ii
    im.set_array(contact_map[structure_index])
    return im,

ani = animation.FuncAnimation(fig, updatefig, frames=contact_map.shape[0], blit=True, repeat=True)
#plt.show()
plt.close()
ani

Time to show how the CA contacts evolves:

In [ ]:
import plotly.express as px

fig = px.imshow(~contact_map, animation_frame=0, binary_string=True, height=600, origin='lower')

fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = list(range(0,contact_map.shape[1],25)),
        ticktext = CA_labels[::25],
        tickangle = 45
    ),
    yaxis = dict(
        tickmode = 'array',
        tickvals = list(range(0,contact_map.shape[1],25)),
        ticktext = CA_labels[::25],
        tickangle = 45
    )
)

fig.show()

## Example 2: Getting distances between geometric centers of different views

As second example let's suppose we have two peptides:

In [ ]:
molsys_A = msm.build.build_peptide('AceAlaNME')
molsys_B = msm.build.build_peptide('AceProNME')
molsys_B = msm.structure.translate(molsys_B, translation='[1.0, 0.0, 0.0] nm')

But each peptide has its own visualization. Notice that this time the views are produced with a different approach than before:

In [ ]:
view1 = nv.show_molsysmt(molsys_A)
view2 = nv.show_molsysmt(molsys_B)

In [ ]:
msm.info(view1)

In [ ]:
msm.info(view2)

If we are interested in the distance between the geometrical center of each peptide, we can merge the views in a new one to extract the observable from a unique molecular system:

In [ ]:
view = msm.merge([view1, view2])

In [ ]:
view.clear()
view.add_licorice()
view

In [ ]:
msm.structure.get_distances(view, selection='molecule_index==0', selection_2='molecule_index==1',
                            group_behavior='geometric center', group_behavior_2='geometric center')

It can be proved looking at the geometric centers that the distance is correct:

In [ ]:
msm.structure.get_geometric_center(view, selection='molecule_index==0')

In [ ]:
msm.structure.get_geometric_center(view, selection='molecule_index==1')

Or we can do it from both views, without merging them in a new one:

In [ ]:
msm.structure.get_distances(view1, group_behavior='geometric center',
                            molecular_system_2=view2, group_behavior_2='geometric center')

:::{seealso}
[User guide > Tools > Basic > Info](../user/tools/basic/info.ipynb): Printing out summary information of a molecular system.    
[User guide > Tools > Basic > Get](../user/tools/basic/get.ipynb):     
[User guide > Tools > Basic > Select](../user/tools/basic/select.ipynb):     
[User guide > Tools > Basic > View](../user/tools/basic/view.ipynb): Showing a molecular system.    
[User guide > Tools > Basic > Merge](../user/tools/basic/merge.ipynb): Merging the elements of different molecular systems.    
[User guide > Tools > Build > Build peptide](../user/tools/build/build_peptide.ipynb):    
[User guide > Tools > Structure > Translate](../user/tools/structure/translate.ipynb):    
[User guide > Tools > Structure > Get distances](../user/tools/structure/get_distances.ipynb):    
[User guide > Tools > Structure > Get geometric center](../user/tools/structure/get_geometric_center.ipynb):    
:::